In [2]:
import pandas as pd
from pyomo.environ import *
from itertools import product
from clean_data import *
import random 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [6]:
route = 'C:/Jupyter/TESIS/instancias/real/testInstance_Amtrak.csv'
start_estation = 'NYP'
commer_class_hierar = {'Y': '1-1','YA': '1-2','YB': '1-3','YC': '1-4','YD': '1-5','YE': '1-6','YF': '1-7','YG': '1-8','JY': '2-1','JA': '2-2','JB': '2-3','JC': '2-4','JD': '2-5','JE': '2-6','JF': '2-7','JG': '2-8'}

origin1, destin, oridest, vagones, periodos, stations, clases, preco, dem, data, anonymo_stat = clean_data(route, start_estation, commer_class_hierar)
origin2 = [0] + origin1
n = len(stations)
cap = 493


In [9]:
# instanciar o modelo
model = AbstractModel()

# Escalares
model.n = Param(initialize = n)
model.Q = Param(initialize = cap)

# conjuntos
model.I2 = Set(initialize = origin2)
model.J = Set(initialize = destin)
model.OD = Set(dimen=2, initialize = oridest)
model.V = Set(initialize = vagones)
model.T = Set(initialize = periodos)
model.VK = Set(model.V, initialize=clases)
def wagons_class(model):
    return ((v,k) for v in clases.keys() for k in clases[v])
model.VKI = Set(dimen=2,initialize=wagons_class)

#parâmetros indexados
model.P = Param(model.OD, model.VKI, initialize = preco)
model.d = Param(model.OD, model.VKI, model.T, initialize = dem)

# variáveis de decisão
model.X = Var(model.I2, model.J, model.VKI, model.T, within = NonNegativeIntegers)
model.Y = Var(model.I2, model.J, model.VKI, model.T, within = NonNegativeIntegers)
model.AU = Var(within = NonNegativeIntegers)

# função objetivo
def fo1(model):
    return sum(model.P[i,j,v,k]*(model.X[i,j,v,k,t]) for i,j in model.OD for v,k in model.VKI for t in model.T)
model.OBJ1 = Objective(rule = fo1, sense = maximize)

# restrições
def rest1(model , _, j):
    if j > 1:
        return sum((model.X[i,j,v,k,t]) for i,_ in model.OD for v,k in model.VKI for t in model.T if i<j ) <= model.Q
    else:
        return Constraint.Skip
model.ConstrainRest1 = Constraint(model.OD, rule = rest1)

def rest2(model , i, _):
    if i < model.n:
        return sum((model.X[i,j,v,k,t]) for _,j in model.OD for v,k in model.VKI for t in model.T if j>i ) <= model.Q
    else:
        return Constraint.Skip
model.ConstrainRest2 = Constraint(model.OD, rule = rest2)

def authorizationCap(model):
    return  sum(model.Y[i,j,v,list(model.VK[v])[0],t] for i,j in model.OD for v in model.VK for t in model.T) <= model.Q
model.ConstrainAuthorizationCap = Constraint(rule = authorizationCap)

def authorization(model, i, j, v, k):
    lista = list(model.VK[v])
    pos_k = lista.index(k)
    lista_sort = sorted(lista, reverse=True)
    if pos_k == 0:
        return  sum(model.Y[i,j,v,lista_sort[pos_k],t] for t in model.T) >=  sum(model.X[i,j,v,lista_sort[pos_k],t] for t in model.T )
    elif pos_k <= len(lista)-1 :
        return  sum(model.Y[i,j,v,lista_sort[pos_k],t] for t in model.T) >=  sum(model.Y[i,j,v,lista_sort[pos_k-1],t] for t in model.T) + sum(model.X[i,j,v,lista_sort[pos_k],t] for t in model.T )
    else:
        return Constraint.Skip
model.ConstrainAuthorization = Constraint(model.OD, model.VKI, rule = authorization)

def authorization(model, i, j, v, k):
    lista = list(model.VK[v])
    if i < j:
        pos_k = lista.index(k)
        sub_list = lista[pos_k:]
        return  sum(model.Y[i,j,v,k,t] for t in model.T) >= sum(model.X[i,j,v,r,t] for r in sub_list for t in model.T )
    else:
        return Constraint.Skip
model.ConstrainAuthorization = Constraint(model.OD, model.VKI, rule = authorization)

def classe(model, i, j, v, k, t):
    lista = list(model.VK[v])
    if k != lista[-1]:
        pos_k = lista.index(k)
        return model.Y[i,j,v,k,t] >= model.Y[i,j,v,lista[pos_k+1],t]
    else:
        return Constraint.Skip
model.ConstrainClasse = Constraint(model.OD, model.VKI, model.T, rule = classe)

def assignment(model, i, j, v, k, t):
    if i < j:
        return model.X[i,j,v,k,t] <= model.d[i,j,v,k,t]
    else:
        return Constraint.Skip
model.ConstrainAssignment = Constraint(model.OD, model.VKI, model.T,rule = assignment)

instance = model.create_instance()
# instance.pprint()

opt = SolverFactory('cplex')
results = opt.solve(instance, tee=True)
# print(results)


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 22.1.1.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2022.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\Wilmer\AppData\Local\Temp\tmp0fgvwq96.cplex.log' open.
CPLEX> Problem 'C:\Users\Wilmer\AppData\Local\Temp\tmpawcu9zt8.pyomo.lp' read.
Read time = 0.06 sec. (3.76 ticks)
CPLEX> Problem name         : C:\Users\Wilmer\AppData\Local\Temp\tmpawcu9zt8.pyomo.lp
Objective sense      : Maximize
Variables            :   20440  [General Integer: 20440]
Objective nonzeros   :    4599
Linear constraints   :   19141  [Less: 19141]
  Nonzeros           :  193450
  RHS nonzeros       :     192

Variables            : Min LB: 0.000000         Max UB: all infinite   
Objective nonzeros   : Min   : 9.90

In [10]:
print('Valor da função objetivo: ', '\033[1m' + '\033[32m' + str(value(instance.OBJ1)) + '\033[0m')

print('')

# print('Valores das variáveis de decisão: ')
lista = []
for i,j in instance.OD:
    for v,k in instance.VKI:
        for t in instance.T:
            # if value(instance.Y[i,j,v,k,t])!=0 or value(instance.X[i,j,v,k,t])!=0:
                # print(i,j,v,k,t, ' - ',value(instance.Y[i,j,v,k,t]), ' - ', value(instance.X[i,j,v,k,t]) )
            lista.append([str(i) + '-' + str(j),v, k, t, value(instance.P[i,j,v,k]), value(instance.d[i,j,v,k,t]) , value(instance.X[i,j,v,k,t]), value(instance.Y[i,j,v,k,t]) ])

a = pd.DataFrame(lista, columns=['o-d','Vagon','classe','Periodo','preco','demanda','Assignments','Authorizations'])
a.to_excel('prueba_m2.xlsx', index=False)

Valor da função objetivo:  21774.000000000004

